In [1]:
# 관련 라이브러리 불러오기
import pandas as pd
import os
import math
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import f1_score, confusion_matrix
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
# 데이터 파일경로를 찾고 변수 file_path에 저장
file_path = '/content/gdrive/MyDrive/nlp/Corpus/Input_Data'

label_list = os.listdir(file_path)

In [ ]:
## input data 파일명 긁어오기
txt_list = []
for label in label_list:
    temp_list = os.listdir(file_path + '/' + label)
    temp_list = [file for file in temp_list if file.endswith(".txt")]
    txt_list.append(temp_list)

In [4]:
## 총 문서의 수
total_doc = 0
for text in txt_list:
    total_doc += len(text)

In [5]:
##빈도가 높은 형태소 5000개 확인을 위한 딕셔너리
pos_dic = {}

##문서당 명사류인 형태소 저장
text_nn = []

##문서당 형태소 수
text_len = []

##몇개의 문서에서 해당 형태소가 등장했는지 확인 위한 딕셔너리
idf_dic = {}

##test에서 쓰기 위함
IDF = []

In [6]:
for i in range(len(label_list)):
    ##label 별로 문서 관리 위함
    text_nn.append([])
    text_len.append([])
    for j in range(len(txt_list[i])):
        temp = pd.read_csv(file_path + '/' + label_list[i] + '/' + txt_list[i][j], sep="\t", encoding="utf8", names=['word', 'pos'])
        ## TF 구할준비
        nn_pos = []
        temp_len = 0
        ##"여가부/NNG+,/SP"와 같이 품사 태깅된 내용 가져
        for pos_list in temp['pos']:
            ##nan 같이 빈 공백 줄이 포함된 경우 처리 위함
            if type(pos_list) == str:
                ## '+'로 이뤄진 규칙이 있어 "여가부/NNG", ",/SP"로 나눔
                for pos in pos_list.split('+'):
                    ## '/'로 이뤄진 규칙이 있어 "여가부", "NNG"로 나눔
                    for item in pos.split('/'):
                        temp_len += 1
                        ## 명사류 형태소
                        if 'NNG' in item:
                            ## 딕셔너리에 있는지 확인하고 count
                            try:
                                pos_dic[pos] += 1
                            except:
                                pos_dic[pos] = 1
                            nn_pos.append(pos)
                        ## 명사류 형태소
                        elif 'NNP' in item:
                            ## 딕셔너리에 있는지 확인하고 count
                            try:
                                pos_dic[pos] += 1
                            except:
                                pos_dic[pos] = 1
                            nn_pos.append(pos)
        text_nn[i].append(nn_pos)
        text_len[i].append(temp_len)

        ## IDF 구할 준비 완료
        for idf in list(set(nn_pos)):
            try:
                idf_dic[idf] +=1
            except:
                idf_dic[idf] = 1

sorted_dict = dict(sorted(pos_dic.items(), key=lambda item: item[1], reverse=True))
top_5000 = dict(list(sorted_dict.items())[:5000])

file_name = sum(txt_list, [])
train_df = pd.DataFrame(index=[file_name], columns=[list(top_5000.keys())])
train_df = train_df.reset_index()
train_df["label"] = 99

for pos in top_5000.keys():
    IDF.append(math.log(total_doc / idf_dic[pos]))

num = 0
for i in range(len(train_df)):
    if label_list[num] in train_df.loc[i]['level_0']:
        train_df.loc[i, 'label'] = num
    else:
        num +=1
        train_df.loc[i, 'label'] = num

chk = 0
for i in range(len(text_nn)):
    for j in range(len(text_nn[i])):
        for nn in list(set(text_nn[i][j])):
            if nn in list(top_5000.keys()):
                cnt = text_nn[i][j].count(nn)
                TF = cnt / text_len[i][j]
                p_chk = list(top_5000.keys()).index(nn)
                train_df.loc[chk, nn] = TF * IDF[p_chk]
        chk += 1

train_df = train_df.fillna(0)

folder_path = '/content/gdrive/MyDrive/nlp/202340211_Dohyoungkim/Input_Data/'
for index, row in train_df.iterrows():
    # 각 행의 첫 번째 feature를 파일명으로 사용
    file_name = row['level_0']
    label = row['label']

    # 파일로 저장 (탭으로 열 구분)
    file_path = f'{folder_path}' + label_list[label] + '/' + f'{file_name}'
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write('\t'.join(map(str, row[1:].tolist())))

file_path = '/content/gdrive/MyDrive/nlp/202340211_Dohyoungkim/all_train_features.txt'
train_df.iloc[:, 1:].to_csv(file_path, sep='\t', index=False, header=False, line_terminator='\n')

<ipython-input-6-80d6d8134a3d>:80: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  train_df.iloc[:, 1:].to_csv(file_path, sep='\t', index=False, header=False, line_terminator='\n')


In [7]:
## Test data 만들기
file_path = '/content/gdrive/MyDrive/nlp/Corpus/Test_Data'

label_list = os.listdir(file_path)

txt_list = [] ## input data 파일명 긁어오기
for label in label_list:
    temp_list = os.listdir(file_path + '/' + label)
    temp_list = [file for file in temp_list if file.endswith(".txt")]
    txt_list.append(temp_list)

file_name = sum(txt_list, [])
test_df = pd.DataFrame(index=[file_name], columns=[list(top_5000.keys())])
test_df = test_df.reset_index()
test_df["label"] = 99

num = 0
for i in range(len(test_df)):
    if label_list[num] in test_df.loc[i]['level_0']:
        test_df.loc[i, 'label'] = num
    else:
        num +=1
        test_df.loc[i, 'label'] = num

text_nn = [] ##문서당 명사류인 형태소 저장
text_len = [] ##문서당 형태소 수
for i in range(len(label_list)):
    text_nn.append([]) ##label 별로 문서 관리 위함
    text_len.append([])
    for j in range(len(txt_list[i])):
        temp = pd.read_csv(file_path + '/' + label_list[i] + '/' + txt_list[i][j], sep="\t", encoding="utf8", names=['word', 'pos'])
        ## TF 구할준비
        nn_pos = []
        temp_len = 0
        for pos_list in temp['pos']: ##"여가부/NNG+,/SP"와 같이 품사 태깅된 내용 가져
            if type(pos_list) == str: ##nan 같이 빈 공백 줄이 포함된 경우 처리 위함
                for pos in pos_list.split('+'): ## '+'로 이뤄진 규칙이 있어 "여가부/NNG", ",/SP"로 나눔
                    for item in pos.split('/'): ## '/'로 이뤄진 규칙이 있어 "여가부", "NNG"로 나눔
                        temp_len += 1
                        if 'NNG' in item: ## 명사류 형태소
                            nn_pos.append(pos)
                        elif 'NNP' in item: ## 명사류 형태소
                            nn_pos.append(pos)
        text_nn[i].append(nn_pos)
        text_len[i].append(temp_len)

chk = 0
for i in range(len(text_nn)):
    for j in range(len(text_nn[i])):
        for nn in list(set(text_nn[i][j])):
            if nn in list(top_5000.keys()):
                cnt = text_nn[i][j].count(nn)
                TF = cnt / text_len[i][j]
                p_chk = list(top_5000.keys()).index(nn)
                test_df.loc[chk, nn] = TF * IDF[p_chk]
        chk += 1

test_df = test_df.fillna(0)

In [8]:
folder_path = '/content/gdrive/MyDrive/nlp/202340211_Dohyoungkim/Test_Data/'
for index, row in test_df.iterrows():
    # 각 행의 첫 번째 feature를 파일명으로 사용
    file_name = row['level_0']
    label = row['label']

    # 파일로 저장 (탭으로 열 구분)
    file_path = f'{folder_path}' + label_list[label] + '/' + f'{file_name}'
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write('\t'.join(map(str, row[1:].tolist())))

file_path = '/content/gdrive/MyDrive/nlp/202340211_Dohyoungkim/all_test_features.txt'
test_df.iloc[:, 1:].to_csv(file_path, sep='\t', index=False, header=False, line_terminator='\n')

<ipython-input-8-5aa0f35e27d6>:13: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  test_df.iloc[:, 1:].to_csv(file_path, sep='\t', index=False, header=False, line_terminator='\n')


In [9]:
#SVM 학습
X_train = train_df.iloc[:, 1:-1]  # 첫 번째 feature와 마지막 feature를 제외한 feature
y_train = train_df['label']

X_test = test_df.iloc[:, 1:-1]
y_test = test_df['label']

In [10]:
# SVM 모델 학습
svm_model = svm.SVC(kernel='sigmoid', decision_function_shape='ovr', random_state=42)
print(svm_model)

SVC(kernel='sigmoid', random_state=42)


In [11]:
svm_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='sigmoid', random_state=42)

In [12]:
# 테스트 데이터 예측
y_pred = svm_model.predict(X_test)

# F1 스코어 계산 및 출력
f1 = f1_score(y_test, y_pred, average='micro')
print(f'F1 Score(micro): {f1}')

f1 = f1_score(y_test, y_pred, average='macro')
print(f'F1 Score(macro): {f1}')

f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score(weighted): {f1}')

F1 Score(micro): 0.7299999999999999
F1 Score(macro): 0.7318519777045036
F1 Score(weighted): 0.7310484155151148


In [13]:
# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

Confusion Matrix:
[[14  1  0  0  0  0  0  0  0]
 [ 1 13  0  0  1  0  0  0  0]
 [ 1  2  3  0  4  0  0  0  0]
 [ 4  0  0  6  0  0  0  0  0]
 [ 1  2  3  0  4  0  0  0  0]
 [ 2  0  0  0  0  8  0  0  0]
 [ 1  0  0  0  0  0  9  0  0]
 [ 1  1  0  0  0  0  0  8  0]
 [ 1  0  0  0  1  0  0  0  8]]
